In [1]:
import yfinance as yf
import pandas as pd 
from sqlalchemy import create_engine

In [2]:
engine = create_engine('sqlite:///test000_Database.db')

In [3]:
df = yf.download('IBKR',start='2023-01-01',end='2023-12-01')
df

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-01-03,72.610001,72.900002,71.330002,71.379997,70.970016,1016200
2023-01-04,71.500000,72.309998,71.470001,72.120003,71.705772,967700
2023-01-05,71.620003,72.300003,70.989998,71.839996,71.427383,527700
2023-01-06,72.269997,73.870003,71.830002,73.650002,73.226990,502700
2023-01-09,74.379997,77.050003,73.690002,74.830002,74.400215,1126300
...,...,...,...,...,...,...
2023-11-24,80.529999,81.830002,80.389999,81.800003,81.618233,435500
2023-11-27,81.550003,81.629997,80.220001,80.849998,80.670341,816300
2023-11-28,80.800003,80.860001,76.349998,76.410004,76.240211,2675700


In [4]:
df.to_sql('IBKR',engine)

230

In [5]:
pd.read_sql('IBKR',engine)

,Date,Open,High,Low,Close,Adj Close,Volume
0,2023-01-03,72.610001,72.900002,71.330002,71.379997,70.970016,1016200
1,2023-01-04,71.500000,72.309998,71.470001,72.120003,71.705772,967700
2,2023-01-05,71.620003,72.300003,70.989998,71.839996,71.427383,527700
3,2023-01-06,72.269997,73.870003,71.830002,73.650002,73.226990,502700
4,2023-01-09,74.379997,77.050003,73.690002,74.830002,74.400215,1126300
...,...,...,...,...,...,...,...
225,2023-11-24,80.529999,81.830002,80.389999,81.800003,81.618233,435500
226,2023-11-27,81.550003,81.629997,80.220001,80.849998,80.670341,816300
227,2023-11-28,80.800003,80.860001,76.349998,76.410004,76.240211,2675700
228,2023-11-29,76.620003,77.279999,72.599998,76.779999,76.609383,2419500


In [6]:
pd.read_sql('SELECT COUNT(*) FROM IBKR',engine)

,COUNT(*)
0,230


In [7]:
max_date = pd.read_sql('SELECT MAX(Date) FROM IBKR',engine).values[0][0]
max_date 

'2023-11-30 00:00:00.000000'

In [8]:
def sql_importer(symbol) :
    max_date = pd.read_sql(f'SELECT MAX(Date) FROM {symbol}',engine).values[0][0]
    print(max_date)
    new_data = yf.download(symbol,start=pd.to_datetime(max_date))
    new_rows = new_data[new_data.index > max_date]
    new_rows.to_sql(symbol, engine, if_exists='append')
    print(str(len(new_rows)) + ' new rows imported to DB')

In [9]:
sql_importer('IBKR')

[*********************100%%**********************]  1 of 1 completed

2023-11-30 00:00:00.000000
72 new rows imported to DB


In [11]:
# import data until today